Import libs

In [1]:
import numpy as np
import pandas as pd  
from matplotlib import pyplot as plt
from sklearn import linear_model
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from sklearn.model_selection import train_test_split
# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
import os
from __init__ import root_dir, data_path, src_path
os.path.abspath(root_dir)
from src.models.utils import compute_error, get_data_for_component, preprocess
%load_ext autoreload
%autoreload 2

Read data

In [2]:
X_numpy = pd.read_csv(os.path.join(data_path,'processed','data_processed.csv'),parse_dates=['datetime'])
print("Vector of", X_numpy.shape)
X_numpy = X_numpy.drop(columns=['failure','Unnamed: 0'])
N, D = X_numpy.shape
print("Vector of", N, "x", D)

Vector of (290642, 42)
Vector of 290642 x 40


Choose scope-target of the model and keep only the corresponding columns

In [3]:
life=True#flag if we want to predict lifespan or not
if life==True:
    X_numpy.dropna(axis=0, inplace=True)
components = ["comp"+str(i)+"_life" for i in range(1,5)]
X0 = get_data_for_component(X_numpy, components[0])

In [4]:
X_numpy.isna().sum()

machineID            0
datetime             0
voltmean_3h          0
rotatemean_3h        0
pressuremean_3h      0
vibrationmean_3h     0
voltsd_3h            0
rotatesd_3h          0
pressuresd_3h        0
vibrationsd_3h       0
voltmean_24h         0
rotatemean_24h       0
pressuremean_24h     0
vibrationmean_24h    0
voltsd_24h           0
rotatesd_24h         0
pressuresd_24h       0
vibrationsd_24h      0
error1count          0
error2count          0
error3count          0
error4count          0
error5count          0
comp1                0
comp2                0
comp3                0
comp4                0
comp1_life           0
comp2_life           0
comp3_life           0
comp4_life           0
age                  0
model_model1         0
model_model2         0
model_model3         0
model_model4         0
comp1_fail           0
comp2_fail           0
comp3_fail           0
comp4_fail           0
dtype: int64

Split and std data 

In [5]:

y, X, X_train_torch, y_train_torch,X_test_torch, X_test, y_test, X_train, y_train, y_std, y_mean=preprocess(X0.copy(),0.1)


Define models, train and test them

In [6]:
X0.columns

Index(['voltmean_3h', 'rotatemean_3h', 'pressuremean_3h', 'vibrationmean_3h',
       'voltsd_3h', 'rotatesd_3h', 'pressuresd_3h', 'vibrationsd_3h',
       'voltmean_24h', 'rotatemean_24h', 'pressuremean_24h',
       'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h', 'pressuresd_24h',
       'vibrationsd_24h', 'error1count', 'error2count', 'error3count',
       'error4count', 'error5count', 'age', 'model_model1', 'model_model2',
       'model_model3', 'model_model4', 'comp1_life'],
      dtype='object')

In [7]:
from sklearn import datasets, linear_model

regr = linear_model.LinearRegression(fit_intercept=False)
regr.fit(X_train_torch, y_train_torch)
preds_lr = regr.predict(X_test_torch)
print(regr.coef_)

mae = np.mean(np.abs(y_test-preds_lr))
print("MAE:", mae)

[-1.3775880e-02  1.0716524e-03 -1.2307315e-03  1.6640450e-03
 -8.6010143e-04 -2.8204415e-03  2.6229597e-04  3.7947111e-04
 -2.5117554e-02 -2.6605227e-03  2.5870181e-03  1.1132961e-03
 -5.3956159e-03 -8.7619363e-04 -1.6128893e-03  5.7711871e-03
 -3.4052096e-02 -3.7997090e-03  6.0530035e-03 -2.7326681e-04
  7.5445324e-03  1.5423427e-02  3.1862695e+02  3.1900397e+02
  4.0285294e+02  3.8920804e+02]
MAE: 0.7432485469172565


A bit high given the range of the $y$ values...

Perhaps we can get a better intuition about the quality of the predictions of the linear regression model if we try to visualize them:

## Neural Network model in Pyro

As mentioned in the lecture's slides, we can obtain complex non-linear function of the inputs by making use of Neural Networks! The following code implements a multi-layer fully-connected NNet in Pyro. Note that, since Pyro is built on top of PyTorch, it is very well suited for building Bayesian Neural Networks and combining them with larger, more complex probabilistic graphical models. This is not the case for STAN! 

We begin by defining the neural network as a `PyroModule` instead of a PyTorch module (i.e. a class that extends ```torch.nn.Module```), as it is customary when using PyTorch for implementing deep neural networks. `PyroModule` is a subclass of `nn.Module`, which allows us to Pyro-ize an existing class (e.g. `linear_layer = PyroModule[nn.Linear](in_dim, out_dim))`) and utilize modules already created in PyTorch. Additionally, priors are placed on the weights by using `PyroSample` as illustrated below. Every neural network in PyTorch has a  `forward` function, that - implementation wise - closely resembles how you coded up Pyro models in last week's notebook.

Make sure that you understand the code above and don't hesitate to ask for clarifications! :-)

We can now perform Bayesian inference in this model (in this case using SVI) to compute the posterior distributions over the neural net weights, as we would normally do for other Pyro models:

In [ ]:
# Define guide function
from src.models.train_model import train_nn

model_FFNN,guide=train_nn("modelFFNN", X_train_torch, y_train_torch)



In [9]:
# Define guide function
from src.models.train_model import train_nn

model_RNN,guide=train_nn("modelRNN", X_train_torch, y_train_torch)



After convergence, we can make predictions for the test set by again leveraging the ```Predictive``` class provided by Pyro:
Let's now calculate the MAE and compare it with the linear regression model from above:

In [ ]:
from src.models.test_model import test_nn, test_nn_beta, mae_test, plot_pred

y_pred=test_nn(model_FFNN,guide,X_test_torch)

mae_test(y_pred,y_test)

We can observe that using the Pyro model above, we were able to fit a Neural Network to our non-linear dataset, and that resulted in a much lower error. For many problems where one cares only about the prediction error this could be the end of the story. However, we will do something different here.

Assume that we are actually interested in investigating how $x_1$ affects $y$ (e.g., imagine $y$ is the price of a house and $x_1$ is the square footage), but not so much about the effect of $x_2$. You could use a NNet to model the relationship of $y$ with both $x_1$ and $x_2$, however, NNets are black-boxes that are pretty hard to interpret. So, we are at a crossroads: we can either have a simple linear model (interpretable!) or a complex neural network (black-box). But can we combine both?? :-)

In order to analyse the effect of $x_1$ of $y$, we will explore the combination of PGMs and (Bayesian) Neural Networks! To do so, we will a assume a model of the following form:

\begin{align}
y = \beta \, x_1 + \mbox{NNet}(x_2) + \epsilon
\end{align}

In other words, we use a simple linear regression for modelling the relationship with $x_1$, and a Bayesian neural network for $x_2$.

Can you modify the Pyro code of the neural network above in order to implement this variation of the model?

Once you finished implementing the model above, run inference on it using SVI:


Change x1 in the model's class to something that has the highest lr weight!!!!

In [ ]:
# Define guide function
from src.models.train_model import train_nn
# Define guide function

model_FFNN_interpretable,guide=train_nn("modelFFNN_interpretable", X_train_torch, y_train_torch)


Let us now have a look at the estimated value for the coefficient $\beta$ over the input dimension $x_1$:

In [ ]:
test_nn_beta(model_FFNN_interpretable,guide,X_test_torch)

y_pred=test_nn(model_FFNN_interpretable,guide,X_test_torch)

mae_test(y_pred,y_test)

Unlike with the linear regression model, we managed to recover the true value of $\beta$ that was used to generate the data ($\beta=2$)!!

What can we conclude from this? It seems that modelling more accuratly the relationship between $x_2$ and $y$ with the NNet, takes away some of the "burden" from the linear part of the model, which can then focus on modelling the effect of $x_1$ better! Makes sense, right?

But that is only part of the "story". Let us now look at the predictions for the testset:

In [ ]:


y_true,y_pre=plot_pred(y_pred,y_test,y_std,y_mean,0)

In [ ]:
corr, mae, rae, rmse, r2, yy_true, ppreds = compute_error(y_true, y_pre, None)
print("CorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" % (corr, mae, rmse, r2))

corr, mae, rae, rmse, r2, yy_true, ppreds = compute_error(y_true, y_pre, 60)
print("CorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" % (corr, mae, rmse, r2))

Uau! That is quite a dramatic reduction in prediction error when compared to the simple linear regression model, and we still managed to keep the interpretability over the effect of $x_1$!!

Let's see if visually we can draw a similar conclusion:

We can see that the composite model (Linear PGM + NNet) predicts almost perfectly the target variable $y$ for the testset! Notice that it managed to capture both the linear trend over the $x_1$ dimension and the complex non-linear trend on the $x_2$ dimension (the one that resulted from the sine function...).

A few more visualizations to analyse the quality of the predictions of the composite model (Linear PGM + NNet) in comparison with the predictions of the linear regression model:

In [ ]:
plt.plot(X_test[:,0], y_test, "rx")
plt.plot(X_test[:,0], preds_lr, 'yo')
plt.plot(X_test[:,0], y_pred, 'bo')
plt.legend(["true (noisy) data points", "linear regression", "NNet predictions SVI"])
plt.show()

In [ ]:
plt.plot(X_test[:,1], y_test, "rx")
plt.plot(X_test[:,1], preds_lr, 'yo')
plt.plot(X_test[:,1], y_pred, 'bo')
plt.legend(["true (noisy) data points", "linear regression", "NNet predictions SVI"])
plt.show()